In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import copy as cp
import os
import os, sys
from collections import defaultdict
from urllib.request import urlopen
import json
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from ipywidgets import widgets
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import random
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.colors import n_colors
from plotly.subplots import make_subplots
init_notebook_mode(connected=True)
import cufflinks as cf
cf.go_offline()
from wordcloud import WordCloud , ImageColorGenerator
from PIL import Image
from sklearn.utils import resample
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_selection import SelectKBest, chi2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset = pd.read_csv('/content/drive/Shareddrives/Group No 1/Aaron Sir Project/LS_2.0.csv')
dataset.head()

,STATE,CONSTITUENCY,NAME,WINNER,PARTY,SYMBOL,GENDER,CRIMINAL\nCASES,AGE,CATEGORY,EDUCATION,ASSETS,LIABILITIES,GENERAL\nVOTES,POSTAL\nVOTES,TOTAL\nVOTES,OVER TOTAL ELECTORS \nIN CONSTITUENCY,OVER TOTAL VOTES POLLED \nIN CONSTITUENCY,TOTAL ELECTORS
0,Telangana,ADILABAD,SOYAM BAPU RAO,1,BJP,Lotus,MALE,52,52.0,ST,12th Pass,"30,99,414","2,31,450",376892,482,377374,25.330684,35.468248,1489790
1,Telangana,ADILABAD,RATHOD RAMESH,0,INC,Hand,MALE,3,52.0,ST,12th Pass,"3,64,91,000","1,53,00,000",314057,181,314238,21.092771,29.534285,1489790
2,Telangana,ADILABAD,Godam Nagesh,0,TRS,Car,MALE,0,54.0,ST,Post Graduate,"1,84,77,888","8,47,000",318665,149,318814,21.399929,29.964370,1489790
3,Telangana,ADILABAD,NOTA,0,NOTA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13030,6,13036,0.875023,1.225214,1489790
4,Uttar Pradesh,AGRA,Satyapal Singh Baghel,1,BJP,Lotus,MALE,5,58.0,SC,Doctorate,"7,42,74,036","86,06,522",644459,2416,646875,33.383823,56.464615,1937690


In [ ]:
dataset.isnull().sum()

STATE                                          0
CONSTITUENCY                                   0
NAME                                           0
WINNER                                         0
PARTY                                          0
SYMBOL                                       245
GENDER                                       245
CRIMINAL\nCASES                              245
AGE                                          245
CATEGORY                                     245
EDUCATION                                    245
ASSETS                                       249
LIABILITIES                                  246
GENERAL\nVOTES                                 0
POSTAL\nVOTES                                  0
TOTAL\nVOTES                                   0
OVER TOTAL ELECTORS \nIN CONSTITUENCY          0
OVER TOTAL VOTES POLLED \nIN CONSTITUENCY      0
TOTAL ELECTORS                                 0
dtype: int64

In [ ]:
dataset[dataset.SYMBOL.isnull()==True]['NAME'].unique()

array(['NOTA'], dtype=object)

In [ ]:
def value_cleaner(x):
    try:
        str_temp = (x.split('Rs')[1].split('\n')[0].strip())
        str_temp_2 = ''
        for i in str_temp.split(","):
            str_temp_2 = str_temp_2+i
        return str_temp_2
    except:
        x = 0
        return x
dataset['ASSETS'] = dataset['ASSETS'].apply((value_cleaner))
dataset['LIABILITIES'] = dataset['LIABILITIES'].apply((value_cleaner))
dataset.head()

,STATE,CONSTITUENCY,NAME,WINNER,PARTY,SYMBOL,GENDER,CRIMINAL\nCASES,AGE,CATEGORY,EDUCATION,ASSETS,LIABILITIES,GENERAL\nVOTES,POSTAL\nVOTES,TOTAL\nVOTES,OVER TOTAL ELECTORS \nIN CONSTITUENCY,OVER TOTAL VOTES POLLED \nIN CONSTITUENCY,TOTAL ELECTORS
0,Telangana,ADILABAD,SOYAM BAPU RAO,1,BJP,Lotus,MALE,52,52.0,ST,12th Pass,0,0,376892,482,377374,25.330684,35.468248,1489790
1,Telangana,ADILABAD,RATHOD RAMESH,0,INC,Hand,MALE,3,52.0,ST,12th Pass,0,0,314057,181,314238,21.092771,29.534285,1489790
2,Telangana,ADILABAD,Godam Nagesh,0,TRS,Car,MALE,0,54.0,ST,Post Graduate,0,0,318665,149,318814,21.399929,29.964370,1489790
3,Telangana,ADILABAD,NOTA,0,NOTA,NaN,NaN,NaN,NaN,NaN,NaN,0,0,13030,6,13036,0.875023,1.225214,1489790
4,Uttar Pradesh,AGRA,Satyapal Singh Baghel,1,BJP,Lotus,MALE,5,58.0,SC,Doctorate,0,0,644459,2416,646875,33.383823,56.464615,1937690


In [ ]:
dataset['ASSETS']

0       0
1       0
2       0
3       0
4       0
       ..
2258    0
2259    0
2260    0
2261    0
2262    0
Name: ASSETS, Length: 2263, dtype: int64

In [ ]:
# rename invalid column names
dataset = dataset.rename(columns={'CRIMINAL\nCASES': 'CRIMINAL_CASES', 'GENERAL\nVOTES': 'GENERAL_VOTES', 'POSTAL\nVOTES': 'POSTAL_VOTES', 'TOTAL\nVOTES': 'TOTAL_VOTES', 'OVER TOTAL ELECTORS \nIN CONSTITUENCY': 'OVER_TOTAL_ELECTORS_IN_CONSTITUENCY', 'OVER TOTAL VOTES POLLED \nIN CONSTITUENCY': 'OVER_TOTAL_VOTES_POLLED_IN_CONSTITUENCY', 'TOTAL ELECTORS': 'TOTAL_ELECTORS'})

In [ ]:
dataset.EDUCATION.replace({'Post Graduate\n':'Post Graduate'},inplace=True)
dataset.EDUCATION.unique()
dataset['EDUCATION'].value_counts()

Post Graduate            503
Graduate                 441
Graduate Professional    336
12th Pass                256
10th Pass                196
8th Pass                  78
Doctorate                 73
Others                    50
Literate                  30
5th Pass                  28
Not Available             22
Illiterate                 5
Name: EDUCATION, dtype: int64

In [ ]:
dataset.EDUCATION.unique()
dataset['EDUCATION'].value_counts()

Post Graduate            503
Graduate                 441
Graduate Professional    336
12th Pass                256
10th Pass                196
8th Pass                  78
Doctorate                 73
Others                    50
Literate                  30
5th Pass                  28
Not Available             22
Illiterate                 5
Name: EDUCATION, dtype: int64

In [ ]:
dataset['ASSETS']=pd.to_numeric(dataset['ASSETS'])
dataset['LIABILITIES']=pd.to_numeric(dataset['LIABILITIES'])
dataset['CRIMINAL_CASES'].replace({np.NaN:0})
dataset['CRIMINAL_CASES'] = pd.to_numeric(dataset['CRIMINAL_CASES'], errors='coerce').fillna(0).astype(np.int64)

In [ ]:
dataset = dataset[dataset['GENDER'].notna()]

In [ ]:
# replace Nil values with 0
dataset['ASSETS'] = dataset['ASSETS'].replace(['Nil', '`', 'Not Available'], '0')
dataset['LIABILITIES'] = dataset['LIABILITIES'].replace(['NIL', '`', 'Not Available'], '0')
dataset['CRIMINAL_CASES'] = dataset['CRIMINAL_CASES'].replace(['Not Available'], '0')

# convert ASSETS, LIABILITIES and CRIMINAL_CASES column values into numeric
dataset['ASSETS'] = dataset['ASSETS'].astype(str).astype(float)
dataset['LIABILITIES'] = dataset['LIABILITIES'].astype(str).astype(float)
dataset['CRIMINAL_CASES'] = dataset['CRIMINAL_CASES'].astype(str).astype(int)
dataset1=cp.copy(dataset)
dataset2=cp.copy(dataset)
dataset3=cp.copy(dataset)

In [ ]:
# label encode categorical columns

lblEncoder_state = LabelEncoder()
lblEncoder_state.fit(dataset['STATE'])
dataset['STATE'] = lblEncoder_state.transform(dataset['STATE'])

lblEncoder_cons = LabelEncoder()
lblEncoder_cons.fit(dataset['CONSTITUENCY'])
dataset['CONSTITUENCY'] = lblEncoder_cons.transform(dataset['CONSTITUENCY'])

lblEncoder_name = LabelEncoder()
lblEncoder_name.fit(dataset['NAME'])
dataset['NAME'] = lblEncoder_name.transform(dataset['NAME'])

lblEncoder_party = LabelEncoder()
lblEncoder_party.fit(dataset['PARTY'])
dataset['PARTY'] = lblEncoder_party.transform(dataset['PARTY'])

lblEncoder_symbol = LabelEncoder()
lblEncoder_symbol.fit(dataset['SYMBOL'])
dataset['SYMBOL'] = lblEncoder_symbol.transform(dataset['SYMBOL'])

lblEncoder_gender = LabelEncoder()
lblEncoder_gender.fit(dataset['GENDER'])
dataset['GENDER'] = lblEncoder_gender.transform(dataset['GENDER'])

lblEncoder_category = LabelEncoder()
lblEncoder_category.fit(dataset['CATEGORY'])
dataset['CATEGORY'] = lblEncoder_category.transform(dataset['CATEGORY'])

lblEncoder_edu = LabelEncoder()
lblEncoder_edu.fit(dataset['EDUCATION'])
dataset['EDUCATION'] = lblEncoder_edu.transform(dataset['EDUCATION'])

In [ ]:
# separate train features and label
y = dataset["WINNER"]
X = dataset.drop(labels=["WINNER"], axis=1)
# split dataset into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)
# train and test knn model
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn.predict(X_test)
print("Testing Accuracy is: ", knn.score(X_test, y_test)*100, "%")

Testing Accuracy is:  88.86138613861387 %


In [ ]:
# scaling values into 0-1 range
scaler = MinMaxScaler(feature_range=(0, 1))
features = [
    'STATE', 'CONSTITUENCY', 'NAME', 'PARTY', 'SYMBOL', 'GENDER', 'CRIMINAL_CASES', 'AGE', 'CATEGORY', 'EDUCATION', 'ASSETS', 'LIABILITIES', 'GENERAL_VOTES', 'POSTAL_VOTES', 'TOTAL_VOTES', 'OVER_TOTAL_ELECTORS_IN_CONSTITUENCY', 'OVER_TOTAL_VOTES_POLLED_IN_CONSTITUENCY', 'TOTAL_ELECTORS']
dataset2[features] = scaler.fit_transform(dataset[features])
# separate train features and label
y = dataset2["WINNER"]
X = dataset2.drop(labels=["WINNER"], axis=1)
# split dataset into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)
# train and test knn model
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn.predict(X_test)
print("Testing Accuracy is: ", knn.score(X_test, y_test)*100, "%")

Testing Accuracy is:  91.83168316831683 %


In [ ]:
dataset1

,STATE,CONSTITUENCY,NAME,WINNER,PARTY,SYMBOL,GENDER,CRIMINAL_CASES,AGE,CATEGORY,EDUCATION,ASSETS,LIABILITIES,GENERAL_VOTES,POSTAL_VOTES,TOTAL_VOTES,OVER_TOTAL_ELECTORS_IN_CONSTITUENCY,OVER_TOTAL_VOTES_POLLED_IN_CONSTITUENCY,TOTAL_ELECTORS
0,Telangana,ADILABAD,SOYAM BAPU RAO,1,BJP,Lotus,MALE,52,52.0,ST,12th Pass,0.0,0.0,376892,482,377374,25.330684,35.468248,1489790
1,Telangana,ADILABAD,RATHOD RAMESH,0,INC,Hand,MALE,3,52.0,ST,12th Pass,0.0,0.0,314057,181,314238,21.092771,29.534285,1489790
2,Telangana,ADILABAD,Godam Nagesh,0,TRS,Car,MALE,0,54.0,ST,Post Graduate,0.0,0.0,318665,149,318814,21.399929,29.964370,1489790
4,Uttar Pradesh,AGRA,Satyapal Singh Baghel,1,BJP,Lotus,MALE,5,58.0,SC,Doctorate,0.0,0.0,644459,2416,646875,33.383823,56.464615,1937690
5,Uttar Pradesh,AGRA,Manoj Kumar Soni,0,BSP,Elephant,MALE,0,47.0,SC,Post Graduate,0.0,0.0,434199,1130,435329,22.466390,37.999125,1937690
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2257,Maharashtra,YAVATMAL-WASHIM,Vaishali Sudhakar Yede,0,PHJSP,Whistle,FEMALE,0,28.0,GENERAL,10th Pass,0.0,0.0,20563,57,20620,1.076097,1.755157,1916185
2258,Maharashtra,YAVATMAL-WASHIM,Anil Jayram Rathod,0,IND,SHIP,MALE,0,43.0,GENERAL,Post Graduate,0.0,0.0,14661,25,14686,0.766419,1.250060,1916185
2259,Telangana,ZAHIRABAD,B.B.PATIL,1,TRS,Car,MALE,18,63.0,GENERAL,Graduate,0.0,0.0,434066,178,434244,28.975369,41.574183,1498666
2260,Telangana,ZAHIRABAD,BANALA LAXMA REDDY,0,BJP,Lotus,MALE,3,47.0,GENERAL,12th Pass,0.0,0.0,138731,216,138947,9.271379,13.302678,1498666


In [ ]:
encoded_edu = []
# iterate through each row in the dataset
for row in dataset1.itertuples():
    education = row.EDUCATION
    if education == "Illiterate":
        encoded_edu.append(0)
    elif education == "Literate":
        encoded_edu.append(1)
    elif education == "5th Pass":
        encoded_edu.append(2)
    elif education == "8th Pass":
        encoded_edu.append(3)
    elif education == "10th Pass":
        encoded_edu.append(4)
    elif education == "12th Pass":
        encoded_edu.append(7)
    elif education == "Graduate":
        encoded_edu.append(8)
    elif education == "Post Graduate":
        encoded_edu.append(9)
    elif education == "Graduate Professional":
        encoded_edu.append(10)
    elif education == "Doctorate":
        encoded_edu.append(11)
    else:
        encoded_edu.append(5)
dataset1['EDUCATION'] = encoded_edu
dataset1['EDUCATION']

0        7
1        7
2        9
4       11
5        9
        ..
2257     4
2258     9
2259     8
2260     7
2261     9
Name: EDUCATION, Length: 2018, dtype: int64

In [ ]:
dataset1['EDUCATION'].value_counts()

9     503
8     441
10    336
7     256
4     196
3      78
11     73
5      72
1      30
2      28
0       5
Name: EDUCATION, dtype: int64

In [ ]:
dataset1['PARTY'].value_counts()

BJP       420
INC       413
IND       201
BSP       163
CPI(M)    100
         ... 
ravp        1
PDP         1
MNF         1
MSHP        1
JDR         1
Name: PARTY, Length: 132, dtype: int64

In [ ]:
# change party of the less frequent parties as Other
# 'BJP','INC','IND','BSP', 'CPI(M)', 'AITC', 'MNM': high frequent
# 'TDP', 'VSRCP', 'SP', 'DMK', 'BJD': medium frequent
dataset1.loc[~dataset1["PARTY"].isin(['BJP','INC','IND','BSP', 'CPI(M)', 'AITC', 'MNM', 'TDP', 'VSRCP', 'SP', 'DMK', 'BJD']), "PARTY"] = "Other"
dataset1['PARTY'].value_counts()

Other     530
BJP       420
INC       413
IND       201
BSP       163
CPI(M)    100
AITC       47
SP         39
MNM        36
TDP        25
DMK        23
BJD        21
Name: PARTY, dtype: int64

In [ ]:
lblEncoder_state = LabelEncoder()
lblEncoder_state.fit(dataset1['STATE'])
dataset1['STATE'] = lblEncoder_state.transform(dataset1['STATE'])

lblEncoder_cons = LabelEncoder()
lblEncoder_cons.fit(dataset1['CONSTITUENCY'])
dataset1['CONSTITUENCY'] = lblEncoder_cons.transform(dataset1['CONSTITUENCY'])

lblEncoder_name = LabelEncoder()
lblEncoder_name.fit(dataset1['NAME'])
dataset1['NAME'] = lblEncoder_name.transform(dataset1['NAME'])

lblEncoder_party = LabelEncoder()
lblEncoder_party.fit(dataset1['PARTY'])
dataset1['PARTY'] = lblEncoder_party.transform(dataset1['PARTY'])

lblEncoder_symbol = LabelEncoder()
lblEncoder_symbol.fit(dataset1['SYMBOL'])
dataset1['SYMBOL'] = lblEncoder_symbol.transform(dataset1['SYMBOL'])

lblEncoder_gender = LabelEncoder()
lblEncoder_gender.fit(dataset1['GENDER'])
dataset1['GENDER'] = lblEncoder_gender.transform(dataset1['GENDER'])

lblEncoder_category = LabelEncoder()
lblEncoder_category.fit(dataset1['CATEGORY'])
dataset1['CATEGORY'] = lblEncoder_category.transform(dataset1['CATEGORY'])

lblEncoder_edu = LabelEncoder()
lblEncoder_edu.fit(dataset1['EDUCATION'])
dataset1['EDUCATION'] = lblEncoder_edu.transform(dataset1['EDUCATION'])
dataset1[['PARTY','EDUCATION']]

,PARTY,EDUCATION
0,2,6
1,6,6
2,9,8
4,2,10
5,3,8
...,...,...
2257,9,4
2258,7,8
2259,9,7
2260,2,6


In [ ]:
# scaling values into 0-1 range
scaler = MinMaxScaler(feature_range=(0, 1))
features = [
    'STATE', 'CONSTITUENCY', 'NAME', 'PARTY', 'SYMBOL', 'GENDER', 'CRIMINAL_CASES', 'AGE', 'CATEGORY', 'EDUCATION', 'ASSETS', 'LIABILITIES', 'GENERAL_VOTES', 'POSTAL_VOTES', 'TOTAL_VOTES', 'OVER_TOTAL_ELECTORS_IN_CONSTITUENCY', 'OVER_TOTAL_VOTES_POLLED_IN_CONSTITUENCY', 'TOTAL_ELECTORS']
dataset1[features] = scaler.fit_transform(dataset1[features])
# separate train features and label
y = dataset1["WINNER"]
X = dataset1.drop(labels=["WINNER"], axis=1)
# split dataset into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)
# train and test knn model
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn.predict(X_test)
print("Testing Accuracy is: ", knn.score(X_test, y_test)*100, "%")

Testing Accuracy is:  92.82178217821783 %


In [ ]:
# Preparing feature values
cons_per_state = {}
voters_per_state = {}
party_winningSeats = {}
party_criminal = {}
party_education = {}
party_totalCandidates_per_cons = {}
party_winningSeats_per_cons = {}
party_criminal_per_cons = {}
party_education_per_cons = {}
voters_per_cons = {}
# group by state
subset = dataset[['STATE', 'CONSTITUENCY', 'TOTAL_ELECTORS']]
gk = subset.groupby('STATE')
# for each state
for name,group in gk:
    # total constituencies per state
    cons_per_state[name] = len(group)
    
    # total voters per state
    voters_per_state[name] = group['TOTAL_ELECTORS'].sum()
# group by party
subset = dataset[['PARTY', 'CONSTITUENCY', 'CRIMINAL_CASES', 'EDUCATION', 'WINNER']]
gk = subset.groupby('PARTY')
# for each party
for name,group in gk:
    # winning seats by party
    party_winningSeats[name] = group[group['WINNER'] == 1.0].shape[0]
    
    # criminal cases by party
    party_criminal[name] = group['CRIMINAL_CASES'].sum()
    
    # education qualification by party (sum of candidates)
    party_education[name] = group['EDUCATION'].sum()
    
    # group by constituency
    gk2 = group.groupby('CONSTITUENCY')
    # for each constituency
    for name2, group2 in gk2:
        key = str(name2)+'_'+str(name)    # cons_party
        
        # total candidates by party in constituency
        party_totalCandidates_per_cons[key] = len(group2)
        
        # party winning seats in the constituency
        party_winningSeats_per_cons[key] = group2[group2['WINNER'] == 1.0].shape[0]
        
        # criminal cases by party in the constituency
        party_criminal_per_cons[key] = group2['CRIMINAL_CASES'].sum()
# education qualification by party in constituency (sum of candidates)
        party_education_per_cons[key] = group2['EDUCATION'].sum()
# Total voters per constituency
subset = dataset[['CONSTITUENCY', 'TOTAL_ELECTORS']]
gk = subset.groupby('CONSTITUENCY')
# for each constituency
for name,group in gk:
    voters_per_cons[name] = len(group)
# Applying feature values
# new feature columns
total_cons_per_state = []
total_voters_per_state = []
total_voters_per_cons = []
winning_seats_by_party = []
criminal_by_party = []
education_by_party = []
total_candidates_by_party_per_cons = []
winning_seats_by_party_per_cons = []
criminal_by_party_per_cons = []
education_by_party_per_cons = []
# iterate through each row in the dataset
for row in dataset.itertuples():
    subkey = str(row.CONSTITUENCY)+'_'+str(row.PARTY)
    total_cons_per_state.append(cons_per_state.get(row.STATE))
    total_voters_per_state.append(voters_per_state.get(row.STATE))
    total_voters_per_cons.append(voters_per_cons.get(row.CONSTITUENCY))
    winning_seats_by_party.append(party_winningSeats.get(row.PARTY))
    criminal_by_party.append(party_criminal.get(row.PARTY))
    education_by_party.append(party_education.get(row.PARTY))
    total_candidates_by_party_per_cons.append(party_totalCandidates_per_cons.get(subkey))
    winning_seats_by_party_per_cons.append(party_winningSeats_per_cons.get(subkey))
    criminal_by_party_per_cons.append(party_criminal_per_cons.get(subkey))
    education_by_party_per_cons.append(party_education_per_cons.get(subkey))
# append columns to dataset
dataset['total_cons_per_state'] = total_cons_per_state
dataset['total_voters_per_state'] = total_voters_per_state
dataset['total_voters_per_cons'] = total_voters_per_cons
dataset['winning_seats_by_party'] = winning_seats_by_party
dataset['criminal_by_party'] = criminal_by_party
dataset['education_by_party'] = education_by_party
dataset['total_candidates_by_party_per_cons'] = total_candidates_by_party_per_cons
dataset['winning_seats_by_party_per_cons'] = winning_seats_by_party_per_cons
dataset['criminal_by_party_per_cons'] = criminal_by_party_per_cons
dataset['education_by_party_per_cons'] = education_by_party_per_cons
dataset

,STATE,CONSTITUENCY,NAME,WINNER,PARTY,SYMBOL,GENDER,CRIMINAL_CASES,AGE,CATEGORY,EDUCATION,ASSETS,LIABILITIES,GENERAL_VOTES,POSTAL_VOTES,TOTAL_VOTES,OVER_TOTAL_ELECTORS_IN_CONSTITUENCY,OVER_TOTAL_VOTES_POLLED_IN_CONSTITUENCY,TOTAL_ELECTORS,total_cons_per_state,total_voters_per_state,total_voters_per_cons,winning_seats_by_party,criminal_by_party,education_by_party,total_candidates_by_party_per_cons,winning_seats_by_party_per_cons,criminal_by_party_per_cons,education_by_party_per_cons
0,31,0,1712,1,26,80,1,52,52.0,2,1,0.0,0.0,376892,482,377374,25.330684,35.468248,1489790,74,127212819,3,300,898,2466,1,1,52,1
1,31,0,1497,0,46,66,1,3,52.0,2,1,0.0,0.0,314057,181,314238,21.092771,29.534285,1489790,74,127212819,3,52,734,2506,1,0,3,1
2,31,0,700,0,119,32,1,0,54.0,2,11,0.0,0.0,318665,149,318814,21.399929,29.964370,1489790,74,127212819,3,9,32,105,1,0,0,11
4,33,1,1788,1,26,80,1,5,58.0,1,4,0.0,0.0,644459,2416,646875,33.383823,56.464615,1937690,251,458868319,3,300,898,2466,1,1,5,4
5,33,1,1118,0,35,49,1,0,47.0,1,11,0.0,0.0,434199,1130,435329,22.466390,37.999125,1937690,251,458868319,3,11,175,816,1,0,0,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2257,19,537,1975,0,85,125,0,0,28.0,0,0,0.0,0.0,20563,57,20620,1.076097,1.755157,1916185,175,321123079,6,0,0,0,1,0,0,0
2258,19,537,184,0,47,99,1,0,43.0,0,11,0.0,0.0,14661,25,14686,0.766419,1.250060,1916185,175,321123079,6,4,131,1011,2,0,0,22
2259,31,538,224,1,119,32,1,18,63.0,0,5,0.0,0.0,434066,178,434244,28.975369,41.574183,1498666,74,127212819,3,9,32,105,1,1,18,5
2260,31,538,249,0,26,80,1,3,47.0,0,1,0.0,0.0,138731,216,138947,9.271379,13.302678,1498666,74,127212819,3,300,898,2466,1,0,3,1


In [ ]:
# scaling values into 0-1 range
scaler = MinMaxScaler(feature_range=(0, 1))
features=dataset.columns
dataset[features] = scaler.fit_transform(dataset[features])
# separate train features and label
y = dataset["WINNER"]
X = dataset.drop(labels=["WINNER"], axis=1)
# split dataset into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)
# train and test knn model
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn.predict(X_test)
print("Testing Accuracy is: ", knn.score(X_test, y_test)*100, "%")

Testing Accuracy is:  98.51485148514851 %


In [ ]:
dataset.drop(labels=["NAME"], axis=1, inplace=True)
dataset.drop(labels=["SYMBOL"], axis=1, inplace=True)
dataset.drop(labels=["POSTAL_VOTES"], axis=1, inplace=True)
dataset.drop(labels=["GENERAL_VOTES"], axis=1, inplace=True)

In [ ]:
# apply SelectKBest class to extract top most features
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(X, y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)
# concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns, dfscores], axis=1)
featureScores.columns = ['Specs', 'Score']
print(featureScores.nlargest(30, 'Score'))

                                      Specs       Score
25          winning_seats_by_party_per_cons  737.914087
21                   winning_seats_by_party  322.311048
16  OVER_TOTAL_VOTES_POLLED_IN_CONSTITUENCY  285.347141
15      OVER_TOTAL_ELECTORS_IN_CONSTITUENCY  262.177373
14                              TOTAL_VOTES  216.937788
12                            GENERAL_VOTES  216.138799
22                        criminal_by_party   91.167396
13                             POSTAL_VOTES   65.126864
23                       education_by_party   40.390369
24       total_candidates_by_party_per_cons   13.938257
3                                     PARTY    7.258766
20                    total_voters_per_cons    5.302656
4                                    SYMBOL    4.128283
8                                  CATEGORY    4.047031
7                                       AGE    2.077768
18                     total_cons_per_state    0.481673
27              education_by_party_per_cons    0

In [ ]:
X.drop(labels=["TOTAL_ELECTORS"],axis=1, inplace=True)
X.drop(labels=["STATE"], axis=1, inplace=True)
X.drop(labels=["CONSTITUENCY"], axis=1, inplace=True)
X.drop(labels=["GENDER"], axis=1, inplace=True)
X.drop(labels=["criminal_by_party_per_cons"], axis=1, inplace=True)
X.drop(labels=["total_voters_per_state"], axis=1, inplace=True)
X.drop(labels=["CRIMINAL_CASES"], axis=1, inplace=True)
X.drop(labels=["total_cons_per_state"], axis=1, inplace=True)
X.drop(labels=["EDUCATION"], axis=1, inplace=True)
X.drop(labels=["education_by_party_per_cons"], axis=1, inplace=True)
X.drop(labels=["AGE"], axis=1, inplace=True)

In [ ]:
# separate train features and label
y = dataset["WINNER"]
# split dataset into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)
# train and test knn model
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
knn.predict(X_test)
print("Testing Accuracy is: ", knn.score(X_test, y_test)*100, "%")

Testing Accuracy is:  99.5049504950495 %
